# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

Eu tenho que baixar com requests somente a parte que me interessa (div id="conteudo" class="outros") <br>
Ou usar o bs4 pra retirar somente essa parte do todo

O código abaixo recebe e cria um arquivo html com os dados da página de listagens.

In [7]:
#import requests
#
#url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
#r = requests.get(url)
#
#open('estoques.html', 'wb').write(r.content)

O código abaixo (em andamento) serve para limpar o html e eventualmente retirar os links das listagens em pdf

In [8]:
#from bs4 import BeautifulSoup
#
#with open('estoques.html') as fp:
#    soup = BeautifulSoup(fp, 'html.parser')
#
#conteudo = soup.find(id="conteudo")
#conteudo.find_all('strong')
#conteudo.find_all('a')
#for link in conteudo.find_all('a'):
#    print(link.get('href'))

Ao longo do tempo o formato de cada entrada de listagem mudou, então o ideal seria criar uma lista com as listagens já existentes e ir adicionando com base na ultima listagem. O problema é se mudarem o formato de divulgação de novo. acho que vou focar somente na parte de banco de dados que á mais útil e confiável, por enquanto.

## Segunda parte: a criação do banco de dados

In [2]:
import pandas as pd
import os


listas_base = []
caminho = './dados/ListagemSoloCriado/XLSX/estq-max-01.xlsx'
sheet = pd.ExcelFile('./dados/ListagemSoloCriado/XLSX/estq-max-01.xlsx').sheet_names
print(sheet)
df_estq_01 = pd.read_excel(io='./dados/ListagemSoloCriado/XLSX/estq-max-01.xlsx', sheet_name='Table 1', header=1)

df_estq_01.head(5)

['Table 1', 'Table 2', 'Table 3']


,MZ,UEU,QRT,Unnamed: 3
0,1,2,1,AE
1,1,2,3,AE
2,1,2,5,AE
3,1,4,3,AE
4,1,4,5,1388.5667


Para cada um dos arquivos, tem que puxar a lista de sheets e adicionar cada uma delas na base de dados

In [5]:
os.listdir("./dados/ListagemSoloCriado/XLSX")

['estq-max-01.xlsx',
 'estq-max-02.xlsx',
 'estq-max-03.xlsx',
 'estq-max-04.xlsx',
 'estq-max-05.xlsx',
 'estq-max-06.xlsx',
 'estq-max-07.xlsx',
 'estq-max-08.xlsx',
 'estq-max-09.xlsx',
 'estq-max-10.xlsx',
 'estq-max-11.xlsx',
 'estq-max-12.xlsx',
 'estq-max-13.xlsx',
 'estq-max-14.xlsx',
 'estq-max-15.xlsx',
 'estq-max-16.xlsx']

Esta função junta as tabelas de um arquivo

In [7]:
def join_sheet(file):
    """Receives a excel file path and
    returns a dataframe with his sheets
    appended."""

    sheet_list = pd.ExcelFile(file).sheet_names
    df_list = []
    for sheet in sheet_list:
        df_list.append(pd.read_excel(io=file, sheet_name=sheet, header=1))
    listagem = pd.concat(objs=df_list, ignore_index=True)
    colunas = listagem.columns.to_list()
    colunas[-1] = 'estq_max_' + file[-7:-5]
    listagem.columns = colunas

    return listagem


In [8]:
estq_01 = join_sheet(caminho)
estq_01



,MZ,UEU,QRT,estq_max_01
0,1,2,1,AE
1,1,2,3,AE
2,1,2,5,AE
3,1,4,3,AE
4,1,4,5,1388.5667
...,...,...,...,...
6264,8,84,21,0.0
6265,8,84,23,0.0
6266,8,84,25,0.0
6267,8,84,27,0.0


**Feito!**

Agora a função que junta MZUEUQRT

In [1]:
#def mzueuqrt(listagem):
#    """Recebe uma listagem e adiciona
#    a coluna MZUEUQRT"""
#listagem["MZUEUQRT"] = int(str(listagem["MZ"]) + str(listagem["UEU"]) + str(listagem["QRT"]))
#
#return listagem
    

NameError: name 'listagem' is not defined

Agora uma função que recebe um diretório que contém várias listas de excel, checa qual a última planinha adicionada pelo programa (baseado no nome padronizado), e se a planilha não tiver sido adicionada ao banco de dados, adiciona-a. Retorna uma lista contendo vários dataframes

In [13]:
def bd_creator():
    files = os.listdir("./dados/ListagemSoloCriado/XLSX")
    bd_estq = []
    estq_list = open("./dados/estq_list.txt", 'ra')
    #if estq_list == "":


    for planilha in files:
        bd_estq.append(join_sheet(planilha))
    
    return bd_estq


In [15]:
bd_estq = bd_creator()

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'c:\\Users\\m_fos\\Documents\\GitHub\\solo-criado-bddados\\ListagemSoloCriado\\XLSX'

In [ ]:
for df in bd_estq:
    print(df.head(5), "\n")

   MZ  UEU  QRT estq_max_01
0   1    2    1          AE
1   1    2    3          AE
2   1    2    5          AE
3   1    4    3          AE
4   1    4    5   1388.5667 

   MZ  UEU  QRT estq_max_02
0   1    2    3          AE
1   1    2    5          AE
2   1    4    3          AE
3   1    4    5   1388.5667
4   1    4    7   3136.8377 

   MZ  UEU  QRT estq_max_03
0   1    2    3          AE
1   1    2    5          AE
2   1    4    3          AE
3   1    4    5   1388.5667
4   1    4    7   3136.8377 

   MZ  UEU  QRT estq_max_04
0   1    2    3          AE
1   1    2    5          AE
2   1    4    3          AE
3   1    4    5   1388.5667
4   1    4    7   3136.8377 

   MZ  UEU  QRT estq_max_05
0   1    2    3          AE
1   1    2    5          AE
2   1    4    3          AE
3   1    4    5   1388.5667
4   1    4    7   3136.8377 

   MZ  UEU  QRT estq_max_06
0   1    2    3          AE
1   1    2    5          AE
2   1    4    3          AE
3   1    4    5   1388.5667
4   1    4

Coleta Feita! Agora para o próximo passo: